In [ ]:
# prototypage.ipynb - Version avec embeddings locaux

# ========== 1. IMPORTATION DES LIBRAIRIES ==========
import os
import sys
import psycopg2
from psycopg2.extras import RealDictCursor
import numpy as np
import pandas as pd
import json
from typing import List, Dict, Any
import re
from pathlib import Path
from sentence_transformers import SentenceTransformer  # <-- MODÈLE LOCAL
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# ========== 2. CONFIGURATION DES CHEMINS ==========
current_dir = Path.cwd()
project_root = current_dir.parent
data_dir = project_root / "data"
txt_dir = data_dir / "TRANS_TXT"
env_file = project_root / ".env"

print(f"📁 Dossier projet: {project_root}")
print(f"📁 Données: {txt_dir}")

# ========== 3. CHARGER LE MODÈLE D'EMBEDDING LOCAL ==========
def load_embedding_model():
    """Charger un modèle d'embedding local"""
    print("🔧 Chargement du modèle d'embedding local...")
    try:
        # Modèle français léger et performant
        model = SentenceTransformer('dangvantuan/sentence-camembert-base')
        print(f"✅ Modèle chargé: {model}")
        print(f"   Dimensions: {model.get_sentence_embedding_dimension()}")
        return model
    except Exception as e:
        print(f"❌ Erreur chargement modèle: {e}")
        # Alternative
        try:
            model = SentenceTransformer('all-MiniLM-L6-v2')
            print(f"✅ Modèle alternatif chargé")
            return model
        except:
            return None

embedding_model = load_embedding_model()

# ========== 4. CONNEXION À POSTGRESQL ==========
def connect_postgres():
    try:
        conn = psycopg2.connect(
            dbname="rag_chatbot",
            user="postgres",
            password="samah",
            host="localhost",
            port="5432"
        )
        print("✅ Connexion PostgreSQL réussie")
        return conn
    except Exception as e:
        print(f"❌ Erreur PostgreSQL: {e}")
        return None

conn = connect_postgres()

# ========== 5. CRÉATION/MISE À JOUR DES TABLES ==========
def setup_database():
    if not conn:
        return
    
    try:
        cur = conn.cursor()
        
        # Activer pgvector
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        
        # Table documents - ajuster la dimension selon le modèle
        cur.execute("""
            CREATE TABLE IF NOT EXISTS documents (
                id SERIAL PRIMARY KEY,
                file_name VARCHAR(300),
                content TEXT NOT NULL,
                embedding vector(384),  -- 384 pour all-MiniLM-L6-v2
                metadata JSONB,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        # Table chat_history
        cur.execute("""
            CREATE TABLE IF NOT EXISTS chat_history (
                id SERIAL PRIMARY KEY,
                user_query TEXT,
                bot_response TEXT,
                sources TEXT[],
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        conn.commit()
        cur.close()
        print("✅ Base de données prête")
        
    except Exception as e:
        print(f"❌ Erreur base de données: {e}")

if conn:
    setup_database()

# ========== 6. CHARGEMENT DES FICHIERS ==========
def load_text_files(max_files=10):
    if not txt_dir.exists():
        print(f"❌ Dossier introuvable: {txt_dir}")
        return []
    
    txt_files = list(txt_dir.glob("*.txt"))
    if not txt_files:
        print("❌ Aucun fichier .txt trouvé")
        return []
    
    print(f"📁 {len(txt_files)} fichiers .txt trouvés")
    
    documents = []
    for i, file_path in enumerate(txt_files[:max_files]):
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
            
            # Nettoyage
            content = re.sub(r'\s+', ' ', content).strip()
            
            if len(content) > 50:
                documents.append({
                    'id': i + 1,
                    'file_name': file_path.name,
                    'content': content,
                    'file_path': str(file_path),
                    'length': len(content)
                })
                print(f"  ✓ {file_path.name} ({len(content)} caractères)")
                
        except Exception as e:
            print(f"  ✗ {file_path.name}: {e}")
    
    print(f"\n📊 {len(documents)} documents chargés")
    return documents

documents = load_text_files(max_files=5)

# ========== 7. GÉNÉRATION D'EMBEDDINGS LOCAUX ==========
def generate_local_embedding(text: str):
    """Générer un embedding avec le modèle local"""
    if not embedding_model:
        print("❌ Modèle d'embedding non chargé")
        return None
    
    try:
        # Tronquer si trop long
        if len(text) > 1000:
            text = text[:1000]
        
        # Générer l'embedding
        embedding = embedding_model.encode(text)
        
        # Convertir en liste pour PostgreSQL
        return embedding.tolist()
    
    except Exception as e:
        print(f"❌ Erreur génération embedding: {e}")
        return None

def add_embeddings_to_docs(docs: List[Dict]):
    """Ajouter les embeddings aux documents"""
    if not embedding_model:
        print("❌ Impossible de générer les embeddings")
        return docs
    
    print("\n🔧 Génération des embeddings locaux...")
    
    for i, doc in enumerate(docs):
        print(f"  Document {i+1}/{len(docs)}: {doc['file_name']}")
        
        embedding = generate_local_embedding(doc['content'])
        if embedding is not None:
            doc['embedding'] = embedding
            print(f"    ✓ Embedding généré ({len(embedding)} dimensions)")
        else:
            doc['embedding'] = None
            print(f"    ✗ Échec embedding")
    
    return docs

# Ajouter les embeddings
if documents and embedding_model:
    documents = add_embeddings_to_docs(documents)

# ========== 8. INSERTION DANS POSTGRESQL ==========
def insert_documents(docs: List[Dict]):
    if not conn:
        print("❌ Pas de connexion PostgreSQL")
        return 0
    
    inserted = 0
    skipped = 0
    
    print("\n💾 Insertion dans la base...")
    
    try:
        cur = conn.cursor()
        
        for doc in docs:
            if not doc.get('embedding'):
                skipped += 1
                continue
            
            try:
                # Vérifier si existe déjà
                cur.execute(
                    "SELECT id FROM documents WHERE file_name = %s",
                    (doc['file_name'],)
                )
                
                if cur.fetchone():
                    skipped += 1
                    print(f"  ⏭️  {doc['file_name']} (déjà présent)")
                else:
                    # Insérer
                    cur.execute("""
                        INSERT INTO documents (file_name, content, embedding, metadata)
                        VALUES (%s, %s, %s, %s)
                    """, (
                        doc['file_name'],
                        doc['content'],
                        doc['embedding'],
                        json.dumps({
                            'source': 'UBS_TXT',
                            'length': doc['length'],
                            'embedding_type': 'local'
                        })
                    ))
                    inserted += 1
                    print(f"  ✓ {doc['file_name']} (inséré)")
                    
            except Exception as e:
                print(f"  ✗ {doc['file_name']}: {e}")
        
        conn.commit()
        cur.close()
        
    except Exception as e:
        print(f"❌ Erreur insertion: {e}")
        conn.rollback()
    
    print(f"\n📊 Résultat: {inserted} insérés, {skipped} ignorés")
    return inserted

# Insérer les documents
if documents:
    insert_documents(documents)

# ========== 9. RECHERCHE DE SIMILARITÉ ==========
def search_similar(query: str, limit: int = 3):
    """Rechercher documents similaires"""
    if not conn or not embedding_model:
        print("❌ Connexion ou modèle manquant")
        return []
    
    try:
        # Embedding de la requête
        query_embedding = generate_local_embedding(query)
        if not query_embedding:
            return []
        
        cur = conn.cursor(cursor_factory=RealDictCursor)
        
        # Recherche
        cur.execute("""
            SELECT 
                id,
                file_name,
                content,
                embedding <=> %s as distance,
                metadata
            FROM documents
            WHERE embedding IS NOT NULL
            ORDER BY distance ASC
            LIMIT %s
        """, (query_embedding, limit))
        
        results = cur.fetchall()
        cur.close()
        
        # Formater
        formatted = []
        for i, row in enumerate(results):
            similarity = 1 - float(row['distance'])
            formatted.append({
                'rank': i + 1,
                'file': row['file_name'],
                'content': row['content'][:500] + "..." if len(row['content']) > 500 else row['content'],
                'full_content': row['content'],
                'similarity': round(similarity, 3)
            })
        
        return formatted
        
    except Exception as e:
        print(f"❌ Erreur recherche: {e}")
        return []

# ========== 10. GÉNÉRATION DE RÉPONSE (avec Gemini ou local) ==========
def generate_response(query: str, context_docs: List[Dict]):
    """Générer une réponse avec le contexte"""
    if not context_docs:
        return "Je n'ai pas trouvé d'informations pertinentes dans ma base de données."
    
    # Préparer le contexte
    context_parts = []
    for i, doc in enumerate(context_docs):
        context_parts.append(f"[Document {i+1}: {doc['file']}]\n{doc['full_content'][:800]}")
    
    context_text = "\n\n".join(context_parts)
    
    # OPTION 1: Avec Gemini (si disponible)
    try:
        from dotenv import load_dotenv
        load_dotenv(env_file)
        api_key = os.getenv("GEMINI_API_KEY")
        
        if api_key:
            import google.generativeai as genai
            genai.configure(api_key=api_key)
            
            prompt = f"""Tu es un assistant en santé numérique. Utilise le contexte pour répondre.

CONTEXTE:
{context_text}

QUESTION: {query}

RÉPONSE (en français, basée uniquement sur le contexte):"""
            
            model = genai.GenerativeModel('gemini-pro')
            response = model.generate_content(prompt, generation_config={'temperature': 0.3})
            return response.text
    except:
        pass  # Passe à l'option locale
    
    # OPTION 2: Réponse locale simple (fallback)
    prompt = f"""Question: {query}

Contexte disponible:
{context_text}

En te basant sur ces documents, réponds de manière concise:"""
    
    # Réponse basique
    doc_summaries = []
    for doc in context_docs:
        doc_summaries.append(f"• {doc['file']} (pertinence: {doc['similarity']:.1%})")
    
    return f"D'après les documents trouvés:\n" + "\n".join(doc_summaries) + \
           f"\n\nRéponse synthétique: Les documents traitent de sujets liés à la requête '{query}'."

# ========== 11. PIPELINE RAG COMPLET ==========
def rag_pipeline(query: str, top_k: int = 3):
    """Pipeline RAG complet"""
    print(f"\n🔍 Requête: '{query}'")
    print("-" * 40)
    
    # 1. Recherche
    print("📚 Recherche de documents...")
    similar_docs = search_similar(query, limit=top_k)
    
    if not similar_docs:
        return "❌ Aucun document pertinent trouvé."
    
    print(f"✅ {len(similar_docs)} documents trouvés:")
    for doc in similar_docs:
        print(f"  {doc['rank']}. {doc['file']} (score: {doc['similarity']})")
    
    # 2. Génération
    print("\n🤖 Génération de la réponse...")
    response = generate_response(query, similar_docs)
    
    # 3. Sauvegarde historique
    if conn:
        try:
            cur = conn.cursor()
            sources = [doc['file'] for doc in similar_docs]
            cur.execute("""
                INSERT INTO chat_history (user_query, bot_response, sources)
                VALUES (%s, %s, %s)
            """, (query, response, sources))
            conn.commit()
            cur.close()
            print("💾 Conversation sauvegardée")
        except Exception as e:
            print(f"⚠️  Erreur historique: {e}")
    
    return response

# ========== 12. STATISTIQUES ==========
def show_stats():
    if not conn:
        return
    
    try:
        cur = conn.cursor()
        
        cur.execute("SELECT COUNT(*) as total FROM documents")
        total = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) as with_emb FROM documents WHERE embedding IS NOT NULL")
        with_emb = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM chat_history")
        queries = cur.fetchone()[0]
        
        print("\n📊 STATISTIQUES")
        print("-" * 20)
        print(f"📄 Documents: {total}")
        print(f"🔧 Avec embeddings: {with_emb}")
        print(f"💬 Requêtes historisées: {queries}")
        
        cur.close()
        
    except Exception as e:
        print(f"❌ Erreur stats: {e}")

# ========== 13. DÉMONSTRATION ==========
def run_demo():
    print("\n" + "="*50)
    print("🤖 DÉMONSTRATION CHATBOT RAG")
    print("="*50)
    
    test_queries = [
        "santé numérique",
        "formation",
        "recherche",
        "université"
    ]
    
    for query in test_queries:
        print(f"\n❓ Question: {query}")
        response = rag_pipeline(query, top_k=2)
        print(f"\n💬 Réponse:\n{response[:300]}...")
        print("-" * 50)

# ========== 14. INTERFACE INTERACTIVE ==========
def interactive_mode():
    print("\n" + "="*50)
    print("💬 MODE INTERACTIF")
    print("Commands: stats, demo, quit")
    print("="*50)
    
    while True:
        try:
            user_input = input("\n👤 Vous: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() == 'quit':
                print("👋 Au revoir!")
                break
            elif user_input.lower() == 'stats':
                show_stats()
                continue
            elif user_input.lower() == 'demo':
                run_demo()
                continue
            
            response = rag_pipeline(user_input, top_k=3)
            print(f"\n🤖 Assistant: {response}")
            
        except KeyboardInterrupt:
            print("\n👋 Interrompu")
            break
        except Exception as e:
            print(f"❌ Erreur: {e}")

# ========== 15. EXÉCUTION PRINCIPALE ==========
print("\n" + "="*50)
print("CHATBOT RAG - EMBEDDINGS LOCAUX")
print("="*50)

print("\n🔍 ÉTAT:")
print(f"  • Modèle embedding: {'✅ Chargé' if embedding_model else '❌ Absent'}")
print(f"  • PostgreSQL: {'✅ Connecté' if conn else '❌ Absent'}")
print(f"  • Documents chargés: {len(documents)}")

show_stats()

# Menu
print("\n" + "="*50)
print("MENU:")
print("1. Mode interactif")
print("2. Démonstration")
print("3. Quitter")

choice = input("\n👉 Choix (1-3): ").strip()

if choice == "1":
    interactive_mode()
elif choice == "2":
    run_demo()
elif choice == "3":
    print("👋 Au revoir!")
else:
    print("❌ Choix invalide")

if conn:
    conn.close()
    print("\n✅ Connexions fermées")

📁 Dossier projet: c:\Chatbot-RAG
📁 Données: c:\Chatbot-RAG\data\TRANS_TXT
🔧 Chargement du modèle d'embedding local...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Modèle chargé: SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'CamembertModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
   Dimensions: 768
✅ Connexion PostgreSQL réussie
✅ Base de données prête
📁 41 fichiers .txt trouvés
  ✓ 017_00000012.txt (2318 caractères)
  ✓ 018_00000013.txt (409 caractères)
  ✓ 019_00000014.txt (687 caractères)
  ✓ 020_00000015.txt (511 caractères)
  ✓ 022_00000017.txt (1068 caractères)

📊 5 documents chargés

🔧 Génération des embeddings locaux...
  Document 1/5: 017_00000012.txt
    ✓ Embedding généré (768 dimensions)
  Document 2/5: 018_00000013.txt
    ✓ Embedding généré (768 dimensions)
  Document 3/5: 019_00000014.txt
    ✓ Embedding généré (768 d

In [ ]:
# Dans une cellule du notebook
def update_table_dimension():
    """Mettre à jour la table pour les embeddings de 768 dimensions"""
    try:
        conn = psycopg2.connect(
            dbname="rag_chatbot",
            user="postgres",
            password="samah",
            host="localhost",
            port="5432"
        )
        cur = conn.cursor()
        
        # Supprimer et recréer la table avec la bonne dimension
        cur.execute("DROP TABLE IF EXISTS documents CASCADE")
        
        # Recréer avec 768 dimensions
        cur.execute("""
            CREATE TABLE documents (
                id SERIAL PRIMARY KEY,
                file_name VARCHAR(300),
                content TEXT NOT NULL,
                embedding vector(768),  -- 768 dimensions pour Camembert
                metadata JSONB,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        conn.commit()
        cur.close()
        conn.close()
        print("✅ Table mise à jour pour 768 dimensions")
        
    except Exception as e:
        print(f"❌ Erreur: {e}")

# Exécutez cette fonction
update_table_dimension()

In [ ]:
# Test direct du chatbot
def test_chatbot():
    print("🤖 TEST DU CHATBOT RAG")
    print("="*50)
    
    # Questions de test
    test_questions = [
        "Qu'est-ce que la santé numérique ?",
        "Parle-moi des formations",
        "Quelles recherches sont menées ?",
        "Qui sont les enseignants ?"
    ]
    
    for question in test_questions:
        print(f"\n❓ Question: {question}")
        print("-" * 30)
        
        # Recherche
        results = search_similar(question, limit=2)
        if not results:
            print("❌ Aucun résultat trouvé")
            continue
            
        print(f"📚 Documents trouvés ({len(results)}):")
        for doc in results:
            print(f"  • {doc['file']} (score: {doc['similarity']:.3f})")
        
        # Génération réponse
        response = generate_response(question, results)
        print(f"\n💬 Réponse:\n{response[:500]}...")
        print("="*50)

# Exécutez le test
test_chatbot()

In [ ]:
def smart_search(query, limit=3, similarity_threshold=0.3):
    """Recherche intelligente avec seuil de similarité"""
    results = search_similar(query, limit=limit*2)  # Chercher plus de résultats
    
    # Filtrer par seuil de similarité
    filtered = [doc for doc in results if doc['similarity'] >= similarity_threshold]
    
    # Limiter au nombre demandé
    return filtered[:limit]

# Test
print("🔍 Recherche intelligente")
query = "santé"
results = smart_search(query, limit=2, similarity_threshold=0.2)

if results:
    print(f"✅ {len(results)} résultats pertinents:")
    for r in results:
        print(f"  • {r['file']} (score: {r['similarity']:.3f})")

In [ ]:
def analyze_documents():
    """Analyser le contenu des documents"""
    print("📊 ANALYSE DES DOCUMENTS")
    print("="*40)
    
    for doc in documents:
        print(f"\n📄 {doc['file_name']}:")
        print(f"   Caractères: {doc['length']}")
        
        # Afficher les 200 premiers caractères
        preview = doc['content'][:200].replace('\n', ' ')
        print(f"   Extrait: {preview}...")
        
        # Mots clés
        words = doc['content'].lower().split()[:10]
        print(f"   Mots: {', '.join(words)}...")

analyze_documents()

In [ ]:
def simple_chat_interface():
    """Interface chat simple"""
    print("\n" + "="*60)
    print("💬 CHATBOT RAG - UBS SANTÉ NUMÉRIQUE")
    print("="*60)
    print("Tapez votre question ou 'quit' pour quitter")
    print("Tapez 'stats' pour les statistiques")
    print("Tapez 'docs' pour voir les documents")
    print("="*60)
    
    while True:
        user_input = input("\n👤 Vous: ").strip()
        
        if user_input.lower() == 'quit':
            print("👋 Au revoir!")
            break
        elif user_input.lower() == 'stats':
            show_stats()
            continue
        elif user_input.lower() == 'docs':
            print("\n📁 Documents dans la base:")
            try:
                conn = connect_postgres()
                cur = conn.cursor()
                cur.execute("SELECT file_name, LENGTH(content) as length FROM documents")
                for row in cur.fetchall():
                    print(f"  • {row[0]} ({row[1]} caractères)")
                cur.close()
                conn.close()
            except:
                print("  Impossible d'accéder à la base")
            continue
        
        # Traiter la question
        print("🔍 Recherche en cours...")
        
        # Recherche intelligente
        results = smart_search(user_input, limit=3, similarity_threshold=0.2)
        
        if not results:
            print("🤖 Assistant: Je n'ai pas trouvé d'informations pertinentes sur ce sujet.")
            continue
        
        print(f"📚 {len(results)} documents pertinents trouvés")
        
        # Générer la réponse
        response = generate_response(user_input, results)
        
        print(f"\n🤖 Assistant: {response}")
        
        # Afficher les sources
        print("\n📎 Sources utilisées:")
        for doc in results:
            print(f"  • {doc['file']} (pertinence: {doc['similarity']:.1%})")

# Lancez l'interface
simple_chat_interface()

In [ ]:
# Pour démarrer directement l'interface chat
simple_chat_interface()